# Convolutional Neural Network

### Importing the libraries

In [1]:
#import tensorflow 2 which now includes keras
#keras includes preprocessing
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

2023-01-23 13:36:11.999044: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
tf.__version__

'2.11.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [3]:
#apply transfrmations to training set ONLY to prevent overfitting
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)
training_set = train_datagen.flow_from_directory(
    'dataset/training_set',
    target_size=(64,64),
    batch_size=32,
    class_mode='binary')

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [4]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = train_datagen.flow_from_directory(
    'dataset/test_set',
    target_size=(64,64),
    batch_size=32,
    class_mode='binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [5]:
cnn = tf.keras.models.Sequential()

2023-01-23 13:36:21.937580: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Step 1 - Convolution

In [6]:
#filters: the number of feature detectors
#kernel_size: the size of the feature detector - num of columns in feature dectector (ex: kernel_size = 3 -> 3x3 feature detectors)
#activation = relu (rectifier activation function)
#only add input_shape for first layer
cnn.add(tf.keras.layers.Conv2D(filters =32, kernel_size=3, activation = 'relu', input_shape=(64,64,3)))

### Step 2 - Pooling

In [7]:
#apply max poolsing - preserves features while reducing size
#take the maximum value of (pool_size x pool_size) and add to new matrix
#shift by strides
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [8]:
cnn.add(tf.keras.layers.Conv2D(filters =32, kernel_size=3, activation = 'relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [9]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [10]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [11]:
#units = 1 because of binary output
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [12]:
#adam = gradient descent optimizer
#binary_crossentropy for binary outcome -- if nonbinary classification -> categorical_crossentropy
cnn.compile(optimizer = 'adam', loss= 'binary_crossentropy', metrics= ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [13]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25
250/250 [==============================] - 42s 164ms/step - loss: 0.6710 - accuracy: 0.5839 - val_loss: 0.6315 - val_accuracy: 0.6505
Epoch 2/25
250/250 [==============================] - 33s 133ms/step - loss: 0.6165 - accuracy: 0.6629 - val_loss: 0.5839 - val_accuracy: 0.7015
Epoch 3/25
250/250 [==============================] - 34s 134ms/step - loss: 0.5682 - accuracy: 0.7091 - val_loss: 0.5653 - val_accuracy: 0.7100
Epoch 4/25
250/250 [==============================] - 34s 134ms/step - loss: 0.5298 - accuracy: 0.7365 - val_loss: 0.5678 - val_accuracy: 0.7125
Epoch 5/25
250/250 [==============================] - 34s 135ms/step - loss: 0.5027 - accuracy: 0.7509 - val_loss: 0.5088 - val_accuracy: 0.7665
Epoch 6/25
250/250 [==============================] - 34s 138ms/step - loss: 0.4757 - accuracy: 0.7717 - val_loss: 0.5178 - val_accuracy: 0.7475
Epoch 7/25
250/250 [==============================] - 34s 135ms/step - loss: 0.4665 - accuracy: 0.7757 - val_loss: 0.4886 - val_ac

## Part 4 - Making a single prediction

In [36]:
import numpy as np
import keras.utils as image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_5.jpg', target_size=(64,64))
test_image = image.img_to_array(test_image)

#add an extra dimension (batch) because training images were trained in batches
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image/255.0)

1/1 [==============================] - 0s 19ms/step


In [37]:
training_set.class_indices

{'cats': 0, 'dogs': 1}

In [38]:
if result[0][0] > .5:
  prediction = 'dog'
else:
  prediction = 'cat'

print(prediction)

dog
